# Loss and Optimization
## Loss
In the previous lecture we learned to work with data having a set of hyperpaarameters. For linear classifier weights are hyperparameters. For different set of hyperparameters we get different predictions. The questions is , which hyperparameters should we use in our model. In other words, which parameter set is best to use in our model. To get intuition about this question "loss" comes to the picture. In machine learning we want our model to predict as much as perfect. Say we want our a model to classify an image as cat. We want our model that every time it encounters an image it correctly predict as cat or not a cat. When model fails to predict correctly we consider it as loss. Therefore we can define loss as the difference between the predicted value and the actual value for a single example. Now the answer is, we will select that hyperparameter set that results the least loss.

##### Types of loss function
There are different types of loss function. Depending on the problem set and algorithm we use. For linera classifier we use Mean Square Error(MSE).
In general, Given a dataset of examples  {(xi,yi)}i=1 to N ,xi is image and yi is label(integer). Loss over the dataset is a sum of loss over examples.

Loss = L[i] =(f(X[i],W),Y[i])

Loss_for_all = 1/N * Sum(Li(f(X[i],W),Y[i])) 


Now we will discuss Multiclass SVM loss.
##### Multiclass SVM loss:
![alt text](lossfunc.png "loss function")

for a single training example for a specified class level , if the value of specified class level is greater than other values then we consider loss as 0 and a amount of loss otherwise. We compare the difference between specified class value with another value. if the specified class value is greater then we return 0 and the difference+1 otherwise. This type of loss is called **hinge** loss

Suppose 3 training example, 3 classes with some W the scoeres f(x,W)=Wx are
for demonstration we consider an 2d array. each value of the inner array contains value for cat,car and frog. for each array first,second and third arrays are

cat,car, frog=[[3.2,5.1,-1.7],[1.3,4.9,2.0],[2.2,2.5,-3.1]]










